In [81]:
from tensorflow.keras.datasets import mnist

In [82]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [83]:
print(x_train.shape)
print(y_train.shape)

(60000, 28, 28)
(60000,)


In [84]:
x_test.dtype

dtype('uint8')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(x_test[500], cmap=plt.cm.gray)

#Add two layer net in Keras

In [85]:
from tensorflow import keras

In [86]:
model = keras.Sequential([
    keras.layers.Dense(100, activation="sigmoid", input_shape=(784, )),
    keras.layers.Dense(10, activation="softmax")
])

In [ ]:
model

In [87]:
model.compile(optimizer="SGD", loss="categorical_crossentropy", metrics=["accuracy"])

In [88]:
keras.utils.to_categorical(y_train, num_classes=10)

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [89]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2])
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1]*x_test.shape[2])

In [90]:
x_train.shape

(60000, 784)

In [91]:
x_train = x_train/255.0
x_test = x_test/255.0

In [92]:
y_train = keras.utils.to_categorical(y_train, num_classes=10)
y_test =  keras.utils.to_categorical(y_test, num_classes=10)

In [93]:
y_train[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [94]:
model.fit(x_train, y_train, epochs=20, batch_size=32)

Epoch 1/20
1875/1875 [==============================] - 3s 1ms/step - loss: 1.4815 - accuracy: 0.6999
Epoch 2/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.7259 - accuracy: 0.8486
Epoch 3/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.5352 - accuracy: 0.8722
Epoch 4/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4555 - accuracy: 0.8842
Epoch 5/20
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4113 - accuracy: 0.8915
Epoch 6/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3829 - accuracy: 0.8960
Epoch 7/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3627 - accuracy: 0.9005
Epoch 8/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3475 - accuracy: 0.9032
Epoch 9/20
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3352 - accuracy: 0.9062
Epoch 10/20
1875/1875 [==============================] - 3s 1ms/step - loss: 0.325

In [95]:
predictions = model.predict(x_test[0:10])

1/1 [==============================] - 0s 45ms/step


In [96]:
import numpy as np

In [97]:
predictions = np.argmax(predictions, axis=1)

In [98]:
predictions

array([7, 2, 1, 0, 4, 1, 4, 9, 6, 9], dtype=int64)

In [99]:
labels = np.argmax(y_test[0:10], axis=1)

In [100]:
labels

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9], dtype=int64)

In [101]:
predictions == labels

array([ True,  True,  True,  True,  True,  True,  True,  True, False,
        True])

#LeNet

In [102]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [121]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import os
import numpy as np
from PIL import Image

class LeNet:
    def __init__(self, batch_size=32, epochs=20):
        self.batch_size = batch_size
        self.epochs = epochs
        self.model = None
        self._create_lenet()
        self._compile()
    

    def _create_lenet(self):
        self.model = Sequential([
            Conv2D(filters=6, kernel_size=(5,5), 
                   activation='sigmoid', input_shape=(28, 28, 1), 
                   padding='same'),
            AveragePooling2D(pool_size=(2, 2), strides=2),
            
            Conv2D(filters=16, kernel_size=(5,5), 
                   activation='sigmoid', 
                   padding='same'),
            AveragePooling2D(pool_size=(2, 2), strides=2),

            Flatten(),

            Dense(120, activation='sigmoid'),
            Dense(84, activation='sigmoid'),
            Dense(10, activation='softmax')
        ])

    def _compile(self):
        if self.model is None:
            print('Error: Create a model first..')
        
        self.model.compile(optimizer='Adam',
                           loss='categorical_crossentropy',
                           metrics=['accuracy'])
        

    def _preprocess(self):
        # load mnist data
        (x_train, y_train), (x_test, y_test) = mnist.load_data()

        # normalize
        x_train = x_train/255.0
        x_test = x_test/255.0

        # add channel dim
        self.x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)  
        self.x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)  

        # one-hot encoding
        self.y_train = to_categorical(y_train, 10)
        self.y_test = to_categorical(y_test, 10)

    def train(self):
        self._preprocess()
        self.model.fit(self.x_train, self.y_train, 
                  batch_size=self.batch_size, 
                  epochs=self.epochs)

    def save(self, model_path_name):
     
        try:
            # Ensure the directory exists
            directory = os.path.dirname(model_path_name)
            if directory and not os.path.exists(directory):
                os.makedirs(directory)
                
            # Save the model as a .keras file
            self.model.save(model_path_name)
            print(f"Model saved to {model_path_name}")
        except Exception as e:
            print(f"Error saving model: {e}")
    
    def load(self, model_path_name):
    
        try:
            # Load the model from the .keras file
            self.model = tensorflow.keras.models.load_model(model_path_name)
            print(f"Model loaded from {model_path_name}")
            return self.model
        except Exception as e:
            print(f"Error loading model: {e}")
            return None

    def predict(self, images):
            """
            Predict the class labels for the input images.

            Args:
                images (list or np.array): A list or array of image data.

            Returns:
                list: Predicted class probabilities or class labels.
            """
            try:
                # If images is a list, convert it to a numpy array
                if isinstance(images, list):
                    images = np.array(images)
                
                # Ensure the images are in the correct format (e.g., resized, normalized)
                # If images are raw (e.g., not preprocessed), preprocess them
                
                # Preprocessing images (e.g., resizing to match input shape of model)
                processed_images = []
                for img in images:
                    # If the image is a PIL image or a numpy array, resize it to the expected input size
                    # Assuming the model was trained on images of size (32, 32, 3)
                    if isinstance(img, np.ndarray):
                        img = image.array_to_img(img)  # Convert numpy array to a PIL image if needed
                    
                    # Resize the image to match model's input size (e.g., 32x32)
                    img = img.resize((32, 32))  # Change the size if your model needs a different size
                    img = image.img_to_array(img)  # Convert PIL image back to a numpy array
                    
                    # Normalize the image (if needed, e.g., values between 0 and 1)
                    img = img / 255.0  # Assuming the model was trained with normalized images

                    # Append to the list of processed images
                    processed_images.append(img)

                # Convert list of images into a numpy array for prediction
                processed_images = np.array(processed_images)

                # Make predictions with the model
                predictions = self.model.predict(processed_images)
                
                # If model outputs class probabilities, you can get the predicted class labels:
                predicted_labels = np.argmax(predictions, axis=-1)
                
                return predicted_labels  # or return predictions if you need probabilities

            except Exception as e:
                print(f"Error making predictions: {e}")
                return None


In [104]:
lenet = LeNet(batch_size=64, epochs=10)

In [105]:
lenet.train()

Epoch 1/10
938/938 [==============================] - 34s 36ms/step - loss: 1.1475 - accuracy: 0.6136
Epoch 2/10
938/938 [==============================] - 36s 38ms/step - loss: 0.2424 - accuracy: 0.9273
Epoch 3/10
938/938 [==============================] - 58s 62ms/step - loss: 0.1571 - accuracy: 0.9523
Epoch 4/10
938/938 [==============================] - 42s 45ms/step - loss: 0.1146 - accuracy: 0.9654
Epoch 5/10
938/938 [==============================] - 45s 48ms/step - loss: 0.0918 - accuracy: 0.9716
Epoch 6/10
938/938 [==============================] - 49s 52ms/step - loss: 0.0774 - accuracy: 0.9761
Epoch 7/10
938/938 [==============================] - 45s 48ms/step - loss: 0.0667 - accuracy: 0.9790
Epoch 8/10
938/938 [==============================] - 46s 49ms/step - loss: 0.0591 - accuracy: 0.9810
Epoch 9/10
938/938 [==============================] - 47s 50ms/step - loss: 0.0523 - accuracy: 0.9835
Epoch 10/10
938/938 [==============================] - 40s 43ms/step - loss: 0.047

In [106]:
x_test = x_test.reshape(-1,28,28,1)

In [107]:
predictions = np.argmax(lenet.model.predict(x_test[0:10]), axis=1)

1/1 [==============================] - 0s 72ms/step


In [109]:
print(predictions)

[7 2 1 0 4 1 4 9 5 9]


In [110]:
labels = np.argmax(lenet.y_test[0:10], axis=1)

In [111]:
print(labels)

[7 2 1 0 4 1 4 9 5 9]


In [112]:
print(predictions == labels)

[ True  True  True  True  True  True  True  True  True  True]


In [119]:
import tensorflow 

In [114]:
model_saver = LeNet()

In [115]:
model_saver.save('Moore_cnn_model.keras')

Model saved to Moore_cnn_model.keras


In [117]:
model_loader = LeNet()

In [120]:
model_load = model_loader.load('Moore_cnn_model.keras')

Model loaded from Moore_cnn_model.keras


In [124]:
from PIL import Image

In [125]:
image_path = 'Custom MNIST Sample/Digit 0/0_0.png'
img = Image.open(image_path)

In [127]:
predicted_class = model_loader.predict([img])

AttributeError: 'LeNet' object has no attribute 'predict'